# Was sich Verbraucher mit der EEG sparen können

* verminderte Netzgebühren für Bezug von EEG Netzebene 7 minus 28 %

* keine Umsatzsteuer auf Strom aus der EEG

* Strompreisbremse hängt am Netto Arbeitspreis

In [66]:
jahresverbrauch = 2900 #kWh
eigenverbrauchsquote = 0.3

# alles netto!
tarifdetails_energieag_mit_strompreisbremse = {
    "cent_pro_kwh": {
        "arbeitspreis": 21.78
    },
    "euro_pro_jahr": {
        "grundpreis": 39.0
    },
    "strompreisbremse": True
}

# alles netto!
tarifdetails_netzooe = {
    "cent_pro_kwh": {
        "verbrauchspreis": 5.12,
        "netzverlustentgeld": 0.59
    },
    "euro_pro_jahr": {
        "grundpreis": 36.0,
        "entgelt_messleistungen": 26.16
    }
}


tarifdetails = {
    "lieferant": tarifdetails_energieag_mit_strompreisbremse,
    "netzbetreiber": tarifdetails_netzooe
}


def kosten_stromhaendler(jahresverbrauch, eigenverbrauchsquote, tarifdetails):
    fixkosten = tarifdetails['euro_pro_jahr']['grundpreis']
    
    eigenverbrauch = jahresverbrauch * eigenverbrauchsquote
    netzverbrauch = jahresverbrauch - eigenverbrauch
    
    verbrauchskosten = netzverbrauch * tarifdetails['cent_pro_kwh']['arbeitspreis'] / 100.0 #EURO
        
    if (tarifdetails['strompreisbremse']):
        reduzierter_arbeitspreis = max(10, tarifdetails['cent_pro_kwh']['arbeitspreis'] - 30.0)
        # print("reduzierter arbeitspreis:", reduzierter_arbeitspreis, "cent/kWh")
        
        reduzierter_netzverbrauch = min(2900, netzverbrauch)
        # print("reduzierter Jahresverbrauch", reduzierter_netzverbrauch)
        
        reduzierte_verbrauchskosten = reduzierter_netzverbrauch * reduzierter_arbeitspreis / 100.0 #EURO
        if (netzverbrauch > 2900):
            reduzierter_arbeitspreis = reduzierter_arbeitspreis * reduzierter_netzverbrauch / 100.0 #EURO
            # print("Gedeckelter Arbeitspreis pro Jahr", verbrauchskosten)
            unreduzierter_arbeitspreis = (netzverbrauch - 2900) * tarifdetails['cent_pro_kwh']['arbeitspreis'] / 100.0 #EURO
            # print("Restlicher Arbeitspreis pro Jahr", unreduzierter_arbeitspreis)
            verbrauchskosten = reduzierter_arbeitspreis + unreduzierter_arbeitspreis
        else:
            verbrauchskosten = reduzierte_verbrauchskosten
                
    
    return fixkosten + verbrauchskosten


def kosten_netzbetreiber(jahresverbrauch, eigenverbrauchsquote, tarifdetails):
    fixkosten = tarifdetails['euro_pro_jahr']['grundpreis']
    fixkosten += tarifdetails['euro_pro_jahr']['entgelt_messleistungen']
    
    eigenverbrauch = jahresverbrauch * eigenverbrauchsquote
    netzverbrauch = jahresverbrauch - eigenverbrauch
    
    # unreduziert
    netzkosten = tarifdetails['cent_pro_kwh']['verbrauchspreis'] * netzverbrauch
    netzkosten += tarifdetails['cent_pro_kwh']['netzverlustentgeld'] * netzverbrauch
    netzkosten = netzkosten / 100.0 # in EURO
    
    # reduziert um 28%
    netzkosten_reduziert = tarifdetails['cent_pro_kwh']['verbrauchspreis'] * eigenverbrauch * (1.0 - 0.28) # reduktion um 28%
    netzkosten_reduziert += tarifdetails['cent_pro_kwh']['netzverlustentgeld'] * eigenverbrauch * (1.0 - 0.28) # reduktion um 28%
    netzkosten_reduziert = netzkosten_reduziert / 100.0  # in EURO
    
    return fixkosten + netzkosten + netzkosten_reduziert



def kosten_eeg(jahresverbrauch, eigenverbrauchsquote):
    # print("Aus der EEG können", jahresverbrauch * eigenverbrauchsquote, "kWh bezogen werden")
    stromkosten = jahresverbrauch * eigenverbrauchsquote * 11.0 / 100.0 # EURO
    mitgliedabeitrag = 20
    return stromkosten + mitgliedabeitrag


def ersparnis(jahresverbrauch, eigenverbrauchsquote, tarifdetails):
    eigenverbrauch = jahresverbrauch * eigenverbrauchsquote
    netzverbrauch = jahresverbrauch - eigenverbrauch
    if eigenverbrauchsquote > 0:
        print("Die EEG verkauft", eigenverbrauch, "kWh der Händler", netzverbrauch, "kWh")
        
    print("Ersparnis bei einem Jahresverbrauch von", jahresverbrauch, "kWh und einer Eigenverbrauchsquote von", eigenverbrauchsquote)
    kosten_ohne_eeg_netto = kosten_stromhaendler(jahresverbrauch, 0, tarifdetails['lieferant']) + kosten_netzbetreiber(jahresverbrauch, 0, tarifdetails['netzbetreiber'])
    kosten_ohne_eeg = kosten_ohne_eeg_netto * 1.2  # 20% USt
    print("Kosten Ohne EEG", kosten_ohne_eeg)

    kosten_mit_eeg_netto = kosten_stromhaendler(jahresverbrauch, eigenverbrauchsquote, tarifdetails['lieferant']) + kosten_netzbetreiber(jahresverbrauch, eigenverbrauchsquote, tarifdetails['netzbetreiber'])
    kosten_mit_eeg = kosten_mit_eeg_netto * 1.2 + kosten_eeg(jahresverbrauch, eigenverbrauchsquote)
    print("Kosten MIT EEG", kosten_mit_eeg)

    
    return kosten_ohne_eeg - kosten_mit_eeg



ersparnis(jahresverbrauch, eigenverbrauchsquote, tarifdetails)

Die EEG verkauft 870.0 kWh der Händler 2030.0 kWh
Ersparnis bei einem Jahresverbrauch von 2900 kWh und einer Eigenverbrauchsquote von 0.3
reduzierter arbeitspreis: 10 cent/kWh
reduzierter Jahresverbrauch 2900
Kosten Ohne EEG 668.1
reduzierter arbeitspreis: 10 cent/kWh
reduzierter Jahresverbrauch 2030.0
Aus der EEG können 870.0 kWh bezogen werden
Kosten MIT EEG 662.708528


5.391472000000022